In [8]:
!pip install pygame


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pygame
import random
import time
import sys
from collections import deque

# Initialize Pygame
pygame.init()

# Define the maps with fruits and exit portals added
map_hard = """
##################################
#P...##.....##......O....X..##...#
#..##..##..##......##..##..##..###
#..##.......F..............##..###
#........##..##...##..##..##...F.#
#..##..##..##......##..##..##..###
#..##...##............##...K.....#
#..##..##..##..##..##..##......###
#....C.......F.......##.........##
#..##..##.X.#..##..##..##..##..###
#....R..........##..........##..E#
##################################
"""

map_medium = """
##################################
#P.........##......X....##.......#
#......##..##..##..##......##..###
#..##...........F............##..#
#......K..............##........F#
#..##..##..##..##..##..##..##..###
#..##...##.........##.......C....#
#......##..##......##......##..###
#.....................##.........#
#..##......##..##......##..##..###
#....R.......X............F....E.#
##################################
"""

map_easy = """
##################################
#P...........##..................#
#..##..##..##......##..##......###
#..........X.................##..#
#........F...........O.........F.#
#..##......##..##......##......###
#..##........##.............##..##
#......##..##......##..##..##..###
#......................X.......E.#
#..##......##..##......##..##..###
#.........R......................#
##################################
"""



# Game constants
CELL_SIZE = 30
WALL_COLOR = (0, 0, 139)      # Dark blue
DOT_COLOR = (255, 255, 255)   # White
POWER_COLOR = (255, 0, 0)     # Red
PACMAN_COLOR = (255, 255, 0)  # Yellow
BACKGROUND_COLOR = (0, 0, 0)  # Black
TEXT_COLOR = (255, 255, 255)  # White
FRUIT_COLOR = (255, 165, 0)   # Orange
BUTTON_COLOR = (70, 130, 180) # Steel blue
BUTTON_HOVER_COLOR = (100, 149, 237) # Cornflower blue
EXIT_COLOR = (0, 255, 0)      # Green for exit portal

# Ghost colors
GHOST_COLORS = {
    'R': (255, 0, 0),      # Red
    'K': (255, 192, 203),  # Pink
    'C': (0, 255, 255),    # Cyan
    'O': (255, 165, 0),    # Orange
}

class Button:
    def __init__(self, x, y, width, height, text, color=BUTTON_COLOR, hover_color=BUTTON_HOVER_COLOR):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.color = color
        self.hover_color = hover_color
        self.is_hovered = False
        self.font = pygame.font.Font(None, 36)
        
    def draw(self, screen):
        color = self.hover_color if self.is_hovered else self.color
        pygame.draw.rect(screen, color, self.rect, border_radius=10)
        pygame.draw.rect(screen, (255, 255, 255), self.rect, 2, border_radius=10)
        
        text_surface = self.font.render(self.text, True, (255, 255, 255))
        text_rect = text_surface.get_rect(center=self.rect.center)
        screen.blit(text_surface, text_rect)
        
    def check_hover(self, pos):
        self.is_hovered = self.rect.collidepoint(pos)
        return self.is_hovered
        
    def is_clicked(self, pos, event):
        if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            return self.rect.collidepoint(pos)
        return False

class PacManGame:
    def __init__(self, map_str=map_easy, difficulty="Easy"):
        self.maze_str = map_str.strip()
        self.maze = [list(row) for row in self.maze_str.split("\n")]
        self.rows, self.cols = len(self.maze), len(self.maze[0])
        self.difficulty = difficulty
        
        # Calculate screen size
        self.screen_width = self.cols * CELL_SIZE
        self.screen_height = self.rows * CELL_SIZE + 80  # Extra space for score and info
        
        # Initialize game state
        self.pacman_pos = None
        self.auto_move = False
        self.desired_direction = None
        self.ghost_positions = {}
        self.ghost_directions = {}
        self.dots = set()
        self.power_ups = set()
        self.fruits = set()
        self.exit_portal = None  # Exit portal position
        
        # Ghost AI personalities
        self.ghost_personalities = {
            'R': 'chaser',    # Red for direct chase
            'K': 'strategic', # Pink for strategic positioning
            'C': 'ambusher',  # Cyan for ambush tactics  
            'O': 'patroller'  # Orange for patrols areas
        }
        
        # Game state
        self.score = 0
        self.lives = 3
        self.game_over = False
        self.victory = False
        self.power_mode = False
        self.power_timer = 0
        self.moves = 0
        self.total_dots = 0
        self.collected_dots = 0
        self.collected_fruits = 0
        
        # Parse the maze
        self.parse_maze()
        
        self.pacman_pixel_pos = [self.pacman_pos[1] * CELL_SIZE, self.pacman_pos[0] * CELL_SIZE]
        self.pacman_target_tile = self.pacman_pos[:]
        self.pacman_speed = 125  # pixels per second
        
        self.ghost_pixel_positions = {
            gid: [x * CELL_SIZE, y * CELL_SIZE]
            for gid, (y, x) in self.ghost_positions.items()
        }
        
        self.initial_pacman_pos = self.pacman_pos[:]
        self.ghost_respawn_cooldowns = {gid: 0.0 for gid in self.ghost_positions}
        self.initial_ghost_positions = {gid: pos[:] for gid, pos in self.ghost_positions.items()}
        self.ghost_targets = {gid: pos[:] for gid, pos in self.ghost_positions.items()}
        self.ghost_fade_alpha = {gid: 255 for gid in self.ghost_positions}
        self.ghost_fade_timer = {gid: 0.0 for gid in self.ghost_positions}
        self.waiting_for_continue = False

        
        # Set victory conditions based on difficulty
        if difficulty == "Easy":
            self.required_dots = max(1, self.total_dots // 4)  # 25% of dots
            self.required_fruits = 2
        elif difficulty == "Medium":
            self.required_dots = max(1, self.total_dots // 2)  # 50% of dots
            self.required_fruits = 1
        else:  # Hard
            self.required_dots = max(1, (self.total_dots * 3) // 4)  # 75% of dots
            self.required_fruits = 0
        
        # PyGame setup
        self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
        pygame.display.set_caption(f"Pac-Man AI Game - {difficulty} Level")
        
        # Load Pac-Man image
        self.pacman_img = pygame.image.load("assets/pacman.png").convert_alpha()
        self.pacman_img = pygame.transform.scale(self.pacman_img, (CELL_SIZE - 4, CELL_SIZE - 4))
        self.rotated_img = self.pacman_img
        
        # Load ghost images
        self.ghost_frightened_img = pygame.image.load("assets/ghost_frightened.png").convert_alpha()
        self.ghost_images = {
            'R': pygame.image.load("assets/ghost_red.png").convert_alpha(),
            'K': pygame.image.load("assets/ghost_pink.png").convert_alpha(),
            'C': pygame.image.load("assets/ghost_cyan.png").convert_alpha(),
            'O': pygame.image.load("assets/ghost_orange.png").convert_alpha(),
        }

        # Scale images to fit cell size
        self.ghost_frightened_img = pygame.transform.scale(self.ghost_frightened_img, (CELL_SIZE - 4, CELL_SIZE - 4))

        for key in self.ghost_images:
            self.ghost_images[key] = pygame.transform.scale(self.ghost_images[key], (CELL_SIZE - 4, CELL_SIZE - 4))
            
        self.pacman_direction = 'd'  # default (d = right, a = left, w = up, s = down)
        self.ghost_directions = {}    # track ghost directions (id → direction)
        
        self.clock = pygame.time.Clock()
        self.font = pygame.font.Font(None, 36)
        self.small_font = pygame.font.Font(None, 24)
        
    def parse_maze(self):
        """Parse maze and find all game elements"""
        for y in range(self.rows):
            for x in range(self.cols):
                cell = self.maze[y][x]
                if cell == 'P':
                    self.pacman_pos = [y, x]
                elif cell in ['R', 'K', 'C', 'O']:
                    self.ghost_positions[cell] = [y, x]
                    self.ghost_directions[cell] = random.choice(['w', 'a', 's', 'd'])
                elif cell == '.':
                    self.dots.add((y, x))
                    self.total_dots += 1
                elif cell == 'X':
                    self.power_ups.add((y, x))
                elif cell == 'F':
                    self.fruits.add((y, x))
                elif cell == 'E':  # Exit portal
                    self.exit_portal = (y, x)
    
    def is_valid_move(self, y, x):
        """Check if position is valid (not a wall)"""
        return 0 <= y < self.rows and 0 <= x < self.cols and self.maze[y][x] != '#'
    
    def move_pacman(self, direction):
        """Set Pac-Man's target tile instead of instantly jumping"""
        if self.game_over or self.victory:
            return

        self.pacman_direction = direction
        dy, dx = 0, 0
        if direction == "up": dy = -1
        elif direction == "down": dy = 1
        elif direction == "left": dx = -1
        elif direction == "right": dx = 1

        y, x = self.pacman_pos
        ny, nx = y + dy, x + dx

        if self.is_valid_move(ny, nx):
            self.pacman_target_tile = [ny, nx]

    def bfs_next_move(self, start, goal):
        """Return the next tile in a BFS shortest path from start → goal."""
        if start == goal:
            return start

        q = deque([start])
        visited = {tuple(start): None}

        while q:
            y, x = q.popleft()

            for dy, dx in [(-1,0),(1,0),(0,-1),(0,1)]:
                ny, nx = y + dy, x + dx
                if not self.is_valid_move(ny, nx):
                    continue

                if (ny, nx) not in visited:
                    visited[(ny, nx)] = (y, x)
                    q.append((ny, nx))

                    if (ny, nx) == tuple(goal):
                        # Reconstruct path backward from goal
                        path = [(ny, nx)]
                        node = (y, x)
                        while node != tuple(start):
                            path.append(node)
                            node = visited[node]
                        path.reverse()
                        return path[0]  # Next step

        return start  # No path found

    
    def get_ghost_move(self, gid):
        gy, gx = self.ghost_positions[gid]
        py, px = self.pacman_pos

        personality = self.ghost_personalities[gid]

        # POWER MODE → FLEE
        if self.power_mode:
            farthest = None
            max_dist = -1
            for y in range(self.rows):
                for x in range(self.cols):
                    if self.is_valid_move(y, x):
                        d = abs(y - py) + abs(x - px)
                        if d > max_dist:
                            max_dist = d
                            farthest = (y, x)
            target = farthest

        # NORMAL BEHAVIOR
        else:
            if personality == "chaser":  # RED
                target = (py, px)

            elif personality == "strategic":  # PINK → 4 tiles ahead
                dy = self.pacman_direction == "up" and -1 or \
                    self.pacman_direction == "down" and 1 or 0
                dx = self.pacman_direction == "left" and -1 or \
                    self.pacman_direction == "right" and 1 or 0
                target = (py + 4*dy, px + 4*dx)

            elif personality == "ambusher":  # CYAN → real Pac-Man behavior
                if random.random() < 0.3:
                    target = (py, px)  # chase directly 30% of the time
                else:
                    target = (py + random.randint(-2, 2), px + random.randint(-2, 2))

            elif personality == "patroller":  # ORANGE → timid
                if random.random() < 0.6:
                    target = (1, 1)  # scatter 60% of the time
                else:
                    target = (py, px)


        # Make sure target is valid
        if not self.is_valid_move(target[0], target[1]):
            target = (py, px)

        # Return NEXT tile only
        next_tile = self.bfs_next_move((gy, gx), target)
        return next_tile

    
    def _chase_ghost(self, ghost_y, ghost_x, target_y, target_x):
        """Move ghost towards target using Manhattan distance"""
        possible_moves = []
        for dy, dx in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            ny, nx = ghost_y + dy, ghost_x + dx
            if self.is_valid_move(ny, nx):
                dist = abs(ny - target_y) + abs(nx - target_x)
                possible_moves.append((dist, ny, nx))
        
        if possible_moves:
            possible_moves.sort()
            return possible_moves[0][1], possible_moves[0][2]
        return ghost_y, ghost_x
    
    def _strategic_move(self, ghost_y, ghost_x):
        """Move to strategic positions"""
        strategic_positions = [
            (1, 1), (1, self.cols-2), (self.rows-2, 1), (self.rows-2, self.cols-2),
            (self.rows//2, self.cols//2)
        ]
        target_y, target_x = random.choice(strategic_positions)
        return self._chase_ghost(ghost_y, ghost_x, target_y, target_x)
    
    def _ambush_move(self, ghost_y, ghost_x, pacman_y, pacman_x):
        """Try to ambush by predicting Pac-Man's movement"""
        if self.dots:
            closest_dot = min(self.dots, key=lambda pos: abs(pos[0]-pacman_y) + abs(pos[1]-pacman_x))
            target_y, target_x = closest_dot
        else:
            target_y, target_x = pacman_y, pacman_x
        return self._chase_ghost(ghost_y, ghost_x, target_y, target_x)
    
    def _patrol_move(self, ghost_id, ghost_y, ghost_x):
        """Patrol in a predefined pattern"""
        directions = ['w', 'a', 's', 'd']
        random.shuffle(directions)
        

        for direction in directions:
            dy, dx = 0, 0
            if direction == 'w': dy = -1
            elif direction == 's': dy = 1
            elif direction == 'a': dx = -1
            elif direction == 'd': dx = 1

            ny, nx = ghost_y + dy, ghost_x + dx
            if self.is_valid_move(ny, nx):
                self.ghost_directions[ghost_id] = direction
                return ny, nx

        # No valid moves — stay still instead of infinite recursion
        return ghost_y, ghost_x
    
    
    def move_ghosts(self, dt):
        """Smoothly move all ghosts based on delta time."""
        speed = 125  # pixels per second

        for ghost_id in self.ghost_positions:
            current_tile = self.ghost_positions[ghost_id]
            target_tile = self.ghost_targets[ghost_id]

            # Convert to pixel coordinates
            current_px = self.ghost_pixel_positions[ghost_id]
            target_px = [target_tile[1] * CELL_SIZE, target_tile[0] * CELL_SIZE]

            dx = target_px[0] - current_px[0]
            dy = target_px[1] - current_px[1]
            dist = (dx**2 + dy**2) ** 0.5

            if dist < 2:
                # Snap to tile and choose a new target
                self.ghost_positions[ghost_id] = target_tile[:]
                new_y, new_x = self.get_ghost_move(ghost_id)
                self.ghost_targets[ghost_id] = [new_y, new_x]
            else:
                # Move smoothly toward target
                step = speed * dt
                if step > dist:
                    step = dist
                nx = current_px[0] + (dx / dist) * step
                ny = current_px[1] + (dy / dist) * step
                self.ghost_pixel_positions[ghost_id] = [nx, ny]


    
    def check_pixel_collisions(self):
        """Check pixel-level collisions between Pac-Man and ghosts."""
        if self.game_over or self.victory:
            return None

        pac_rect = pygame.Rect(
            self.pacman_pixel_pos[0] + 4,
            self.pacman_pixel_pos[1] + 4,
            CELL_SIZE - 8,
            CELL_SIZE - 8
        )

        for ghost_id, pos in self.ghost_pixel_positions.items():
            if self.ghost_respawn_cooldowns.get(ghost_id, 0) > 0:
                continue
            ghost_rect = pygame.Rect(pos[0] + 4, pos[1] + 4, CELL_SIZE - 8, CELL_SIZE - 8)

            if pac_rect.colliderect(ghost_rect):
                if self.power_mode:
                    # Pac-Man eats ghost
                    self.score += 200
                    self.ghost_respawn_cooldowns[ghost_id] = 5.0  # 5-second respawn delay
                    self.ghost_positions[ghost_id] = [-999, -999]
                    self.ghost_pixel_positions[ghost_id] = [-9999, -9999]
                    self.ghost_targets[ghost_id] = [-999, -999]
                    return "ghost_eaten"
                else:
                    # Ghost catches Pac-Man
                    if not hasattr(self, "death_cooldown") or self.death_cooldown <= 0:
                        self.lives -= 1
                        self.death_cooldown = 2.0

                        if self.lives <= 0:
                            self.game_over = True
                        else:
                            self.reset_after_death()
                            self.wait_for_continue()  # <-- Wait for key press before continuing

                        return "pacman_caught"

        return None

    def reset_after_death(self):
        """Reset Pac-Man and ghosts to their starting positions after a death."""
        # Reset Pac-Man
        self.pacman_pos = self.initial_pacman_pos[:]
        self.pacman_pixel_pos = [self.pacman_pos[1] * CELL_SIZE, self.pacman_pos[0] * CELL_SIZE]
        self.pacman_target_tile = self.pacman_pos[:]
        self.auto_move = False
        
        # Reset ghosts
        for gid, pos in self.initial_ghost_positions.items():
            self.ghost_positions[gid] = pos[:]
            self.ghost_pixel_positions[gid] = [pos[1] * CELL_SIZE, pos[0] * CELL_SIZE]
            self.ghost_targets[gid] = pos[:]  # <--- Reset target to starting tile too!

        # Clear death cooldown movement and pause briefly
        self.death_cooldown = 0
        pygame.time.delay(100)
                
    def wait_for_continue(self):
        """Pause the game after death until the player presses a key."""
        waiting = True
        while waiting:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                elif event.type == pygame.KEYDOWN:
                    waiting = False
            self.screen.fill((0, 0, 0))
            text = self.font.render(f"Lives left: {self.lives}", True, (255, 255, 0))
            msg = self.font.render("Press any key to continue...", True, (255, 255, 255))
            self.screen.blit(text, (self.screen_width // 2 - text.get_width() // 2, self.screen_height // 2 - 50))
            self.screen.blit(msg, (self.screen_width // 2 - msg.get_width() // 2, self.screen_height // 2))
            pygame.display.flip()
            self.clock.tick(30)

    
    def respawn_ghost(self, ghost_id):
        """Respawn ghost at random position"""
        start_y, start_x = self.initial_ghost_positions[ghost_id]
        self.ghost_positions[ghost_id] = [start_y, start_x]
        self.ghost_pixel_positions[ghost_id] = [start_x * CELL_SIZE, start_y * CELL_SIZE]
        self.ghost_targets[ghost_id] = [start_y, start_x]
        # Make ghost invisible
        self.ghost_fade_alpha[ghost_id] = 0
        self.ghost_fade_timer[ghost_id] = 1.5  # fade length in seconds
        self.ghost_respawn_cooldowns[ghost_id] = 0.0
    
    def update_ghost_respawns(self, dt):
        for gid in self.ghost_respawn_cooldowns:
            if self.ghost_respawn_cooldowns[gid] > 0:
                self.ghost_respawn_cooldowns[gid] -= dt

                # cooldown finished → respawn ghost
                if self.ghost_respawn_cooldowns[gid] <= 0:
                    self.respawn_ghost(gid)
    
    def update_ghost_fade(self, dt):
        """Fade-in effect for ghosts after respawn."""
        for gid in self.ghost_fade_timer:
            if self.ghost_fade_timer[gid] > 0:
                self.ghost_fade_timer[gid] -= dt

                # Increase alpha gradually (0 → 255)
                t = max(0, self.ghost_fade_timer[gid])
                alpha = 255 - int((t / 1.5) * 255)
                self.ghost_fade_alpha[gid] = max(0, min(255, alpha))


    
    def respawn_pacman(self):
        """Respawn Pac-Man at starting position"""
        for y in range(self.rows):
            for x in range(self.cols):
                if self.maze[y][x] == 'P':
                    self.pacman_pos = [y, x]
                    break
                
    def update_pacman(self, dt):
        """Smoothly interpolate Pac-Man's position per frame"""
        if self.power_mode:
            speed = 175
        else:
            speed = 125
        # Convert target tile to pixel coords
        target_px = [self.pacman_target_tile[1] * CELL_SIZE, self.pacman_target_tile[0] * CELL_SIZE]
        px, py = self.pacman_pixel_pos

        dx = target_px[0] - px
        dy = target_px[1] - py
        dist = (dx**2 + dy**2) ** 0.5

        if dist < 2:
            # Snap to tile and process collection
            self.pacman_pixel_pos = target_px[:]
            if self.pacman_pos != self.pacman_target_tile:
                self.pacman_pos = self.pacman_target_tile[:]
                self._handle_tile_interaction()
            if self.auto_move and self.desired_direction:
                dy, dx = 0, 0
                if self.desired_direction == "up": 
                    self.rotated_img = pygame.transform.rotate(self.pacman_img, 90)
                    dy = -1
                elif self.desired_direction == "down": 
                    self.rotated_img = pygame.transform.rotate(self.pacman_img, -90)
                    dy = 1
                elif self.desired_direction == "left": 
                    self.rotated_img = pygame.transform.flip(self.pacman_img, True, False)
                    dx = -1
                elif self.desired_direction == "right": 
                    self.rotated_img = self.pacman_img
                    dx = 1
                    
                ny, nx = self.pacman_pos[0] + dy, self.pacman_pos[1] + dx
                if self.is_valid_move(ny, nx):
                    self.pacman_target_tile = [ny, nx]
                else:
                    # stop at wall
                    self.auto_move = False
        else:
            step = speed * dt
            if step > dist:
                step = dist
            px += (dx / dist) * step
            py += (dy / dist) * step
            self.pacman_pixel_pos = [px, py]
    
    def _handle_tile_interaction(self):
        y, x = self.pacman_pos

        if (y, x) in self.dots:
            self.dots.remove((y, x))
            self.score += 10
            self.collected_dots += 1

        if (y, x) in self.power_ups:
            self.power_ups.remove((y, x))
            self.score += 50
            self.power_mode = True
            self.power_timer = 7

        if (y, x) in self.fruits:
            self.fruits.remove((y, x))
            self.score += 100
            self.collected_fruits += 1
            
        if self.collected_dots >= self.required_dots:
            self.victory = True
        # Exit portal check
        if self.exit_portal and (y, x) == self.exit_portal:
            self.victory = True
            self.score += 500

    
    def draw(self):
        """Draw the game using PyGame"""
        self.screen.fill(BACKGROUND_COLOR)
        
        # Draw maze elements
        for y in range(self.rows):
            for x in range(self.cols):
                rect = pygame.Rect(x * CELL_SIZE, y * CELL_SIZE, CELL_SIZE, CELL_SIZE)
                
                # Draw walls
                if self.maze[y][x] == '#':
                    pygame.draw.rect(self.screen, WALL_COLOR, rect)
                    pygame.draw.rect(self.screen, (30, 30, 100), rect, 1)  # Wall border
                
                # Draw dots - Make sure dots are visible
                if (y, x) in self.dots:
                    dot_rect = pygame.Rect(
                        x * CELL_SIZE + CELL_SIZE // 2 - 3,
                        y * CELL_SIZE + CELL_SIZE // 2 - 3,
                        6, 6
                    )
                    pygame.draw.ellipse(self.screen, DOT_COLOR, dot_rect)
                
                # Draw power-ups
                if (y, x) in self.power_ups:
                    power_rect = pygame.Rect(
                        x * CELL_SIZE + CELL_SIZE // 2 - 6,
                        y * CELL_SIZE + CELL_SIZE // 2 - 6,
                        12, 12
                    )
                    pygame.draw.ellipse(self.screen, POWER_COLOR, power_rect)
                    pygame.draw.ellipse(self.screen, (255, 255, 255), power_rect, 1)
                
                # Draw fruits
                if (y, x) in self.fruits:
                    fruit_rect = pygame.Rect(
                        x * CELL_SIZE + CELL_SIZE // 2 - 8,
                        y * CELL_SIZE + CELL_SIZE // 2 - 8,
                        16, 16
                    )
                    pygame.draw.ellipse(self.screen, FRUIT_COLOR, fruit_rect)
                    pygame.draw.ellipse(self.screen, (255, 255, 255), fruit_rect, 1)
                
                # Draw exit portal
                if self.exit_portal and y == self.exit_portal[0] and x == self.exit_portal[1]:
                    portal_rect = pygame.Rect(
                        x * CELL_SIZE + 5,
                        y * CELL_SIZE + 5,
                        CELL_SIZE - 10,
                        CELL_SIZE - 10
                    )
                    pygame.draw.ellipse(self.screen, EXIT_COLOR, portal_rect)
                    pygame.draw.ellipse(self.screen, (255, 255, 255), portal_rect, 2)
                    
                    # Draw portal animation (pulsing effect)
                    pulse = (pygame.time.get_ticks() // 200) % 3
                    if pulse == 0:
                        inner_rect = pygame.Rect(
                            x * CELL_SIZE + 10,
                            y * CELL_SIZE + 10,
                            CELL_SIZE - 20,
                            CELL_SIZE - 20
                        )
                        pygame.draw.ellipse(self.screen, (200, 255, 200), inner_rect)
        
        # Draw Pac-Man
        pacman_y, pacman_x = self.pacman_pos
        pacman_rect = pygame.Rect(
            pacman_x * CELL_SIZE + 2,
            pacman_y * CELL_SIZE + 2,
            CELL_SIZE - 4,
            CELL_SIZE - 4
        )
        pacman_color = (0, 0, 255) if self.power_mode else PACMAN_COLOR
        
        # Draw Pac-Man image instead of circle
        pacman_y, pacman_x = self.pacman_pos
        pacman_x_px = self.pacman_pixel_pos[0] + 2
        pacman_y_px = self.pacman_pixel_pos[1] + 2


        self.screen.blit(self.rotated_img, (pacman_x_px, pacman_y_px))
        
        # Draw ghosts using images
        for ghost_id, (ghost_y, ghost_x) in self.ghost_positions.items():
            ghost_img = self.ghost_images.get(ghost_id)

            # Flip horizontally if ghost is moving left
            if self.ghost_directions.get(ghost_id) == "left":
                ghost_img = pygame.transform.flip(ghost_img, True, False)

            # If in power mode, tint blue
            # Choose the correct ghost image based on power mode
            if self.power_mode:
                ghost_img = self.ghost_frightened_img   # frightened sprite
            else:
                ghost_img = self.ghost_images[ghost_id]  # each ghost's normal image

            px, py = self.ghost_pixel_positions[ghost_id]

            # Only draw ghost if cooldown is finished
            if self.ghost_respawn_cooldowns[ghost_id] <= 0:
                ghost_img.set_alpha(self.ghost_fade_alpha[ghost_id])
                self.screen.blit(ghost_img, (px + 2, py + 2))

            
            


        
        # Draw game info
        info_text = f"Score: {self.score} | Lives: {self.lives}"
        if self.power_mode:
            info_text += f" | Power Mode: {round(self.power_timer)}"
        
        text_surface = self.font.render(info_text, True, TEXT_COLOR)
        self.screen.blit(text_surface, (10, self.rows * CELL_SIZE + 10))
        
        # Draw objective info
        if self.difficulty == "Easy":
            objective = f"Objective: Collect {self.required_dots} dots OR reach green exit"
        elif self.difficulty == "Medium":
            objective = f"Objective: Collect {self.required_dots} dots OR reach green exit"
        else:  # Hard
            objective = f"Objective: Collect {self.required_dots} dots OR reach green exit"
            
        objective_surface = self.small_font.render(objective, True, TEXT_COLOR)
        self.screen.blit(objective_surface, (10, self.rows * CELL_SIZE + 35))
        
        # Draw progress
        progress = f"Progress: {self.collected_dots}/{self.required_dots} dots, {self.collected_fruits}/{self.required_fruits} fruits"
        progress_surface = self.small_font.render(progress, True, TEXT_COLOR)
        self.screen.blit(progress_surface, (10, self.rows * CELL_SIZE + 55))
        
        # Draw exit portal hint
        if self.exit_portal:
            exit_hint = "TIP: Find the GREEN EXIT PORTAL to finish level quickly!"
            hint_surface = self.small_font.render(exit_hint, True, (0, 255, 0))
            self.screen.blit(hint_surface, (self.screen_width - 400, self.rows * CELL_SIZE + 10))
        
        # Draw game over or victory message
        if self.game_over:
            game_over_text = self.font.render("GAME OVER! Press R to restart or ESC to quit", True, (255, 0, 0))
            text_rect = game_over_text.get_rect(center=(self.screen_width // 2, self.screen_height // 2))
            self.screen.blit(game_over_text, text_rect)
        elif self.victory:
            victory_text = self.font.render("VICTORY! Press R to restart or ESC to quit", True, (0, 255, 0))
            text_rect = victory_text.get_rect(center=(self.screen_width // 2, self.screen_height // 2))
            self.screen.blit(victory_text, text_rect)
        pygame.display.flip()
        
        if self.waiting_for_continue:
            msg = self.font.render("Press any key to continue", True, (255, 255, 255))
            rect = msg.get_rect(center=(self.screen_width // 2, self.screen_height // 2))
            self.screen.blit(msg, rect)
    
    

    
    def run(self):
        running = True
        clock = pygame.time.Clock()

        while running:
            dt = clock.tick(120) / 1000.0  # delta time in seconds since last frame

            # --- Handle events ---
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        running = False
                    elif event.key == pygame.K_r:
                        self.__init__(self.maze_str, self.difficulty)  # restart game

                    elif event.key in (pygame.K_UP, pygame.K_w):
                        self.desired_direction = "up"
                        self.auto_move = True
                    elif event.key in (pygame.K_DOWN, pygame.K_s):
                        self.desired_direction = "down"
                        self.auto_move = True
                    elif event.key in (pygame.K_LEFT, pygame.K_a):
                        self.desired_direction = "left"
                        self.auto_move = True
                    elif event.key in (pygame.K_RIGHT, pygame.K_d):
                        self.desired_direction = "right"
                        self.auto_move = True
                    if event.type == pygame.KEYDOWN and self.waiting_for_continue:
                        self.waiting_for_continue = False
                        continue  # Skip other input handling this frame
            
            if not self.victory and not self.game_over:
                # --- Update entities ---
                if hasattr(self, "death_cooldown") and self.death_cooldown > 0:
                    self.death_cooldown -= dt

                self.update_pacman(dt) # if you want smooth Pac-Man motion too
                self.move_ghosts(dt)   # smooth ghost motion
                if self.waiting_for_continue:
                    return
                self.check_pixel_collisions()
                
                # --- POWER MODE TIMER ---
                if self.power_mode:
                    self.power_timer -= dt
                    if self.power_timer <= 0:
                        self.power_mode = False
                
                self.update_ghost_respawns(dt)
                self.update_ghost_fade(dt)



            # --- Draw everything ---
            self.draw()

            # --- Flip the display ---
            pygame.display.flip()




def show_level_selection():
    """Show level selection screen"""
    screen = pygame.display.set_mode((600, 400))
    pygame.display.set_caption("Pac-Man AI Game - Level Selection")
    
    # Create buttons
    easy_button = Button(200, 100, 200, 50, "Easy Level")
    medium_button = Button(200, 170, 200, 50, "Medium Level")
    hard_button = Button(200, 240, 200, 50, "Hard Level")
    quit_button = Button(200, 310, 200, 50, "Quit")
    
    buttons = [easy_button, medium_button, hard_button, quit_button]
    
    font = pygame.font.Font(None, 48)
    title = font.render("PAC-MAN AI GAME", True, (255, 255, 0))
    
    running = True
    selected_level = None
    
    while running:
        mouse_pos = pygame.mouse.get_pos()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                selected_level = "quit"
            
            for button in buttons:
                button.check_hover(mouse_pos)
                
                if button.is_clicked(mouse_pos, event):
                    if button == easy_button:
                        selected_level = "Easy"
                    elif button == medium_button:
                        selected_level = "Medium"
                    elif button == hard_button:
                        selected_level = "Hard"
                    elif button == quit_button:
                        selected_level = "quit"
                    running = False
        
        screen.fill((0, 0, 0))
        
        # Draw title
        title_rect = title.get_rect(center=(300, 50))
        screen.blit(title, title_rect)
        
        # Draw buttons
        for button in buttons:
            button.draw(screen)
        
        pygame.display.flip()
    
    return selected_level

def run_pygame_pacman(map_choice=map_easy, difficulty="Easy"):
    """Run the PyGame version of Pac-Man"""
    print(f"Starting Pac-Man Game - Difficulty: {difficulty}")
    print("Controls: Arrow keys to move, R to restart, ESC to quit")
    print("Close the game window to stop the game")
    
    game = PacManGame(map_choice, difficulty)
    game.run()
    print("Game ended. You can run the cell again to play again")

def main():
    """Main game function with level selection"""
    pygame.init()
    
    while True:
        selected_level = show_level_selection()
        
        if selected_level == "quit" or selected_level is None:
            break
        elif selected_level == "Easy":
            run_pygame_pacman(map_easy, "Easy")
        elif selected_level == "Medium":
            run_pygame_pacman(map_medium, "Medium")
        elif selected_level == "Hard":
            run_pygame_pacman(map_hard, "Hard")
    
    pygame.quit()

# Run the game when script is executed
if __name__ == "__main__":
    main()

Starting Pac-Man Game - Difficulty: Hard
Controls: Arrow keys to move, R to restart, ESC to quit
Close the game window to stop the game
Game ended. You can run the cell again to play again
Starting Pac-Man Game - Difficulty: Hard
Controls: Arrow keys to move, R to restart, ESC to quit
Close the game window to stop the game
Game ended. You can run the cell again to play again
Starting Pac-Man Game - Difficulty: Hard
Controls: Arrow keys to move, R to restart, ESC to quit
Close the game window to stop the game
Game ended. You can run the cell again to play again
